In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import Dataset, DataLoader
from SVarM_utils import *
from SVarM_models import Classifier
from scipy.io import loadmat

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from torch.utils.data import Dataset, DataLoader
c=10
classIDs = ["17072500137","17072803272","17073100024","17080900138","17081103274","17081103275","17090400128","17090403276","17090803277","17091203278","17091303279","17091500223"]
class MeshDataset(Dataset):
    def __init__(self, IDs):
        self.IDs = IDs

    def __len__(self):
        return len(self.IDs)

    def __getitem__(self, idx):
        file_path = self.IDs[idx]
        y = file_path.split("/")[1].split("_")[1] + file_path.split("/")[1].split("_")[2]
        y = classIDs.index(y)
        print(y)
        mesh = loadData(file_path.strip())  
        return mesh, torch.tensor([y])

matdata = loadmat("data_splits/COMA_data_splits.mat")
test_data = matdata['test_data'].tolist()
train_data = matdata['train_data'].tolist()
val_dataset = MeshDataset(test_data)
train_dataset = MeshDataset(train_data)

In [3]:
model = Classifier(12)
sum(p.numel() for p in model.parameters() if p.requires_grad)

1980

In [4]:
batch_size =10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: list(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: list(zip(*x)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.005)
epochs = 100

In [6]:
for epoch in range(epochs):
        model.train()
        count = 0
        train_loss = 0.0
        for meshs, y in train_loader: 
            y = torch.stack(y)
            optimizer.zero_grad()
            outputs = model(meshs)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            count=count+1
            
            print(f"Epoch [{count}/{len(train_loader)}] - Train Loss: {loss.item():.4f}", end ="\r")

        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for meshs, y in val_loader:
                y = torch.stack(y)
                outputs = model(meshs)
                loss = criterion(outputs, y)
                val_loss += loss.item()
                # Compute accuracy
                _, predicted = torch.max(outputs, 1)  # Get predicted class (index of max logit)
                total += y.size(0)  # Total number of samples
                correct += (predicted == y).sum().item()  # Count correct predictions
        accuracy = 100 * correct / total 

        val_loss /= len(val_loader)

        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

8
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170908_03277_TA\mouth_middle\mouth_middle.000004.ply
11
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170915_00223_TA\bareteeth\bareteeth.000075.ply
8
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170908_03277_TA\bareteeth\bareteeth.000078.ply
2
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170731_00024_TA\bareteeth\bareteeth.000115.ply
2
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170731_00024_TA\eyebrow\eyebrow.000079.ply
7
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170904_03276_TA\mouth_extreme\mouth_extreme.000030.ply
7
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170904_03276_TA\bareteeth\bareteeth.000058.ply
8
[Open3D WARNING] Read PLY failed: unable to open file: COMA_data\FaceTalk_170908_03277_TA\high_smile\high_smile.000162.ply
6
[


KeyboardInterrupt

